In [ ]:
import sys
sys.path.append("python/")

import sympy as sp
import numdifftools as nd
import numpy as np
import scipy.optimize as opt

import datlib
import symbase
import Z2num
import Z2sym

# Ферромагнетик в симметричном случае

## Выражение для $Z^{(2)}$

### Интеграл

In [ ]:
h, l = Z2sym.h, Z2sym.l
s1, s2 = symbase.SigmaSymbol("\\sigma_1"), symbase.SigmaSymbol("\\sigma_2")
t, u = sp.symbols("t u")

In [ ]:
expr = sp.Integral(sp.exp(t**2/(2*l) + s1*t), (t, 0, h+s2*l))
expr = expr.transform(t, (t-s1*l, t)).expand()
expr = expr.transform(t, (t*sp.sqrt(2*l), t)).factor()
display(expr)

expr = expr.replace(sp.Integral, lambda *args: sp.exp(args[1][2]**2)).expand()
expr = expr*sp.exp(-(h**2+l**2)/(2*l))
expr.expand()

### Производные $\cal{F}^{(2)}$

In [ ]:
expr = s1*s2*Z2sym._sigmas_expr_E_symmetrical*Z2sym._sigmas_expr_cF2_symmetrical
expr = expr.diff(l).expand()
symbase.collect_sigmas(expr, Z2sym._sigmas_terms_symmetrical, [s1, s2])

In [ ]:
display(Z2sym.E_symmetrical(h, l, 1, 1).diff(h).rewrite(sp.exp))
display(Z2sym.E_symmetrical(h, l, 1, 1).rewrite(sp.exp).diff(h))

In [ ]:
display(Z2sym.cF2_symmetrical(h, l).diff(l).rewrite(sp.exp).expand())
display(Z2sym.cF2_symmetrical(h, l).rewrite(sp.exp).diff(l).expand())

### Численная проверка

In [ ]:
h, l = 1, 1.4
print(Z2num.calc_Z2_integrate(0, h, h, l)*np.exp(-2*h-l))
print(Z2num.calc_from_coeffs_symmetrical(h, l).Z2_norm)

In [ ]:
h, l = Z2sym.h, Z2sym.l
cF2_diff_h = sp.lambdify((h, l), Z2sym.cF2_symmetrical(h, l).diff(h), Z2sym.num_module)
cF2_diff_l = sp.lambdify((h, l), Z2sym.cF2_symmetrical(h, l).diff(l), Z2sym.num_module)

args = 1, 1.8
gradient = nd.Gradient(lambda args: Z2num.cF2_symmetrical(*args))(args)
print(gradient[0])
print(cF2_diff_h(*args))
print()
print(gradient[1])
print(cF2_diff_l(*args))

In [ ]:
h, l = Z2sym.h, Z2sym.l
sigmas_args = 1, 1
E_diff_h = sp.lambdify((h, l), Z2sym.E_symmetrical(h, l, *sigmas_args).diff(h), Z2sym.num_module)
E_diff_l = sp.lambdify((h, l), Z2sym.E_symmetrical(h, l, *sigmas_args).diff(l), Z2sym.num_module)

args = 1, 1.8
sigmas_args = list(map(bool, sigmas_args))

gradient = nd.Gradient(lambda args: Z2num.E_symmetrical(*args, *sigmas_args))(args)

print(gradient[0])
print(E_diff_h(*args))
print()
print(gradient[1])
print(E_diff_l(*args))

## Моменты

### Символьные вычисления

In [ ]:
h, l = Z2sym.h, Z2sym.l
Z2_symmetrical = sp.sqrt(2)*(2*sp.pi)**2*Z2sym.cF2_symmetrical(h, l)/(h*sp.sqrt(l))

In [ ]:
m, eta = Z2sym.m, Z2sym.eta
Z2 = Z2_symmetrical
Z2

In [ ]:
expr_m = sp.log(Z2).diff(h)/2
expr_m = expr_m.expand()
display(sp.Eq(m, expr_m))
sp.print_latex(sp.Eq(m, expr_m))

In [ ]:
expr_eta = sp.log(Z2).diff(l)
expr_eta = expr_eta.expand()
display(sp.Eq(eta, expr_eta))
sp.print_latex(sp.Eq(eta, expr_eta))

In [ ]:
_E12, _E1 = Z2sym.E_symmetrical(h, l, 1, 1), Z2sym.E_symmetrical(h, l, 1, 0)
_E12_E1 = sp.solve([m-expr_m, eta-expr_eta], _E12, _E1)
_expr_E12 = _E12_E1[_E12].factor()
_expr_E1 = _E12_E1[_E1].factor()

_expr_cF2_1 = sp.solve(_E12-_expr_E12, Z2sym.cF2_symmetrical(h, l))[0]
_expr_cF2_2 = sp.solve(_E1-_expr_E1, Z2sym.cF2_symmetrical(h, l))[0]

display(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_1))
sp.print_latex(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_1))
display(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_2))
sp.print_latex(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_2))


In [ ]:
subs_to_m_and_eta_symmetrical = {
    Z2sym.E_symmetrical(h, l, 1, 1): _expr_E12,
    Z2sym.E_symmetrical(h, l, 1, 0): _expr_E1,
    Z2sym.cF2_symmetrical(h, l): _expr_cF2_1
}
subs_to_m_and_eta = subs_to_m_and_eta_symmetrical

In [ ]:
expr_eta2 = Z2.diff(l, 2)/Z2
expr_eta2 = expr_eta2.expand()

expr = expr_eta2.subs(subs_to_m_and_eta).expand()
expr = expr.collect([Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1), m, eta], sp.factor)
display(expr)
sp.print_latex(expr)

In [ ]:
expr_m_eta = Z2.diff(h, l)/(2*Z2)
expr_m_eta = expr_m_eta.expand()

expr = expr_m_eta.subs(subs_to_m_and_eta).expand()
expr = expr.collect([Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1), m, eta], sp.factor)
display(expr)
sp.print_latex(expr)

In [ ]:
_E = Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1)
subs_to_m_eta_symmetrical = {
    _E: sp.solve(expr_m_eta.subs(subs_to_m_and_eta)-Z2sym.m_eta, _E)[0]
}
subs_to_m_eta = subs_to_m_eta_symmetrical

In [ ]:
subs_to_upsilon = {Z2sym.m_eta: m*(1-2*Z2sym.upsilon)}

In [ ]:
expr = expr_eta2.subs(subs_to_m_and_eta).subs(subs_to_m_eta).expand()
subs_to_eta2 = {l: sp.solve(expr-Z2sym.eta2, l)[0]}
expr = sp.Eq(l/2*(Z2sym.eta2-1), sp.expand(l/2*(expr.subs(subs_to_upsilon)-1)))

display(expr)
sp.print_latex(expr)

In [ ]:
for moment, expr_moment in zip([m, eta], [expr_m, expr_eta]):
    for arg in [h, l]:
        expr = expr_moment.diff(arg).expand().subs(subs_to_m_and_eta).expand()
        expr = expr.subs(subs_to_m_eta).factor().expand()
        expr = expr.subs(subs_to_eta2).factor().expand()
        display(sp.Eq(sp.Derivative(moment, arg), expr))

### Численная проверка

In [ ]:
def calc_Z2_symmetrical_integrate(args):
    h, l = args
    return Z2num.calc_Z2_integrate(0, h, h, l)

In [ ]:
f_expr_m = sp.lambdify((h, l), expr_m, Z2sym.num_module_norm)
f_expr_eta = sp.lambdify((h, l), expr_eta, Z2sym.num_module_norm)

args = 4, 5
moments = nd.Gradient(calc_Z2_symmetrical_integrate)(args)/calc_Z2_symmetrical_integrate(args)
z2 = Z2num.calc_from_coeffs_symmetrical(*args)

print(moments[0]/2)
print(f_expr_m(*args))
print(z2.m)
print()
print(moments[1])
print(f_expr_eta(*args))
print(z2.eta)

In [ ]:
f_expr_cF2_1 = sp.lambdify((h, l, m, eta), _expr_cF2_1, Z2sym.num_module)
f_expr_cF2_2 = sp.lambdify((h, l, m, eta), _expr_cF2_2, Z2sym.num_module)

args = 1, 1.6
args_moments = f_expr_m(*args), f_expr_eta(*args)

print(Z2num.cF2_symmetrical(*args))
print(f_expr_cF2_1(*args, *args_moments))
print(f_expr_cF2_2(*args, *args_moments))

In [ ]:
f_expr_m_eta = sp.lambdify((h, l), expr_m_eta, Z2sym.num_module_norm)
f_expr_eta2 = sp.lambdify((h, l), expr_eta2, Z2sym.num_module_norm)

args = 3.4, 1.9
moments = nd.Hessian(calc_Z2_symmetrical_integrate)(args)/calc_Z2_symmetrical_integrate(args)
z2 = Z2num.calc_from_coeffs_symmetrical(*args)

print(moments[0, 1]/2)
print(moments[1, 0]/2)
print(f_expr_m_eta(*args))
print(z2.m_eta)
print()
print(moments[1, 1])
print(f_expr_eta2(*args))
print(z2.eta2)

## Предельные случаи 

### $\lambda \rightarrow 0$

In [ ]:
expr = Z2.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 1).factor()
expr = sp.limit(expr, l, 0).expand()
display(expr)

expr.rewrite(sp.sin).factor()

In [ ]:
expr = expr_m.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 2).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

subs_to_m_llb = {
    sp.exp(2*h): sp.solve(expr.expand()-m, sp.exp(2*h))[0]
}

expr.expand().rewrite(sp.tanh).factor().expand()

In [ ]:
expr = expr_eta.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 3).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr.expand().subs(subs_to_m_llb).factor()

In [ ]:
expr = expr_m_eta.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 4).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr = expr.expand().subs(subs_to_m_llb).factor().expand()
display(expr)
sp.print_latex(expr)

In [ ]:
expr = expr_eta2.rewrite(sp.exp).factor()
expr = Z2sym.dawsn_to_series_oo(expr, 5).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr = expr.expand().subs(subs_to_m_llb).factor().expand()
display(expr)
sp.print_latex(expr)

In [ ]:
# expr = expr_m.diff(l).rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 4).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr.expand().subs(subs_to_m_llb).factor().expand()

In [ ]:
# expr = (expr_eta).diff(l).rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 5).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# expr

In [ ]:
# expr = (expr_m_eta/2).diff(l).factor().rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 6).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# display(expr)

# expr = expr/m - (m-2*m**2/h)/m**2*(m - m * m * m - 2 * m * m/h)
# expr = expr.expand()
# display(expr)
# display(expr.replace(m, h/3))

In [ ]:
# expr = expr_eta2.diff(l).factor().rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 7).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# display(expr)
# display(expr.replace(m, h/3))

### $h \rightarrow 0$

In [ ]:
expr = Z2.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h).replace(h, 0).expand()
display(expr)

expr = expr.rewrite(sp.sin).factor()
display(expr)
sp.print_latex(expr)

In [ ]:
expr = expr_m.rewrite(sp.exp).factor()
expr = symbase.LHopital(expr, h, 2)
expr = expr.replace(h, 0)
display(expr)

In [ ]:
expr = expr_eta.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h)
expr = expr.replace(h, 0).factor()
display(expr)

subs_to_eta_prmg = {
    sp.exp(2*l): sp.solve(expr.expand()-eta, sp.exp(2*l))[0]
}

expr = expr.expand().rewrite(sp.tanh).factor().expand()
display(expr)
sp.print_latex(expr)

In [ ]:
expr = ((1-expr_m_eta/expr_m)/2)
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor()

In [ ]:
expr = expr_eta2.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor().expand()

In [ ]:
expr = expr_m/h
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor()

In [ ]:
# expr = (expr_m/h).diff(h).factor()
# expr = symbase.LHopital(expr, h, 5)
# expr = expr.rewrite(sp.exp).replace(h, 0)
# display(expr)

In [ ]:
expr = expr_eta.diff(h).factor()
expr = symbase.LHopital(expr, h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0)
display(expr)

In [ ]:
# expr = ((1-expr_m_eta/expr_m)/2).diff(h).factor()
# expr = symbase.LHopital(expr.factor(), h, 6)
# expr = expr.rewrite(sp.exp).replace(h, 0).factor()
# display(expr)

## Генерация кода C++ для реализации метода Ньютона

In [ ]:
subs = {
    Z2sym.cF2_symmetrical(h, l): sp.Symbol("cF2_norm"),
    Z2sym.E_symmetrical(h, l, 1, 1): sp.Symbol("E12_norm"),
    Z2sym.E_symmetrical(h, l, 1, 0): sp.Symbol("E1_norm"),
    Z2sym.E_symmetrical(h, l, 0, 0): sp.Symbol("E_norm"),
    m: sp.Symbol("m"), eta: sp.Symbol("eta"),
    sp.exp(-2*h): sp.Symbol("_exp_2h"),
    sp.exp(-2*l): sp.Symbol("_exp_2l"),
    l: sp.Symbol("l")
}


def optimize(expr):
    expr = symbase.optimize_pow(expr, {l: "l"})
    expr = expr.subs(subs)
    return expr

In [ ]:
print(symbase.get_ccode(optimize(.5*(1-expr_m_eta/m)).factor()))

## Решение обратной задачи

### Прямая задача

In [ ]:
h = np.logspace(-5, 1, 100)
l = np.logspace(-5, 1, 100)

h, l = np.meshgrid(h, l)

In [ ]:
m, eta = np.empty_like(h), np.empty_like(h)
upsilon, eta2 = np.empty_like(h), np.empty_like(h)
Z2_norm = np.empty_like(m)

for i in range(h.size):
    z2 = Z2num.calc_from_coeffs_symmetrical(h.flat[i], l.flat[i])
    m.flat[i], eta.flat[i] = z2.m, z2.eta
    upsilon.flat[i], eta2.flat[i] = z2.upsilon, z2.eta2
    Z2_norm.flat[i] = z2.Z2_norm

In [ ]:
datlib.np2dat(
    "data/Z2.dat",
    "h l upsilon eta2 Z2_norm".split(),
    np.asarray([
        h, l, upsilon, eta2, Z2_norm
    ])
)

In [ ]:
# %%bash
# gplt3 -pm3d -U 'Z2_norm' data/Z2.dat -ln xy -to data/picts/pict.pdf 

### Обратная задача

In [ ]:
m = np.linspace(0, 1, 100)
zeta = np.linspace(0, 1, 100)

m, zeta = np.meshgrid(m, zeta)
eta = zeta + (1-zeta)*m**2

In [ ]:
p, h, l = np.empty_like(m), np.empty_like(m), np.empty_like(m)
upsilon, eta2, mhj = np.empty_like(m), np.empty_like(m), np.empty_like(m)
psi0, Z2_norm = np.empty_like(m), np.empty_like(m)

for i in range(m.size):
    z2 = Z2num.find_coeffs_symmetrical(m.flat[i], eta.flat[i])
    h.flat[i], l.flat[i] = z2.h, z2.l
    upsilon.flat[i], eta2.flat[i], mhj.flat[i] = z2.upsilon, z2.eta2, z2.mh2
    psi0.flat[i], Z2_norm.flat[i] = z2.psi0, z2.Z2_norm
    p.flat[i] = Z2num.invL(m.flat[i])

In [ ]:
datlib.np2dat(
    "data/Z2.dat",
    "m eta zeta p h l upsilon eta2 mhj psi0 Z2_norm".split(),
    np.asarray([
        m, eta, zeta, p, h, l, upsilon, eta2, mhj, psi0, Z2_norm
    ])
)

In [ ]:
# %%bash
# gplt3 -pm3d -U 'Z2_norm(m,zeta)' data/Z2.dat -to data/picts/pict.pdf 

### Асимптотика при $\left<m\right> \rightarrow 1$

In [ ]:
m = np.linspace(.99, 1, 100)
zeta = np.linspace(.05, 1, 100)

m, zeta = np.meshgrid(m, zeta)
eta = zeta + (1-zeta)*m**2

In [ ]:
h, l = np.empty_like(m), np.empty_like(m)
upsilon, eta2, mhj = np.empty_like(m), np.empty_like(m), np.empty_like(m)
psi0, Z2_norm = np.empty_like(m), np.empty_like(m)

for i in range(m.size):
    z2 = Z2num.find_coeffs_symmetrical_by_scipy(m.flat[i], eta.flat[i])
    h.flat[i], l.flat[i] = z2.h, z2.l
    upsilon.flat[i], eta2.flat[i], mhj.flat[i] = z2.upsilon, z2.eta2, z2.mh2
    psi0.flat[i], Z2_norm.flat[i] = z2.psi0, z2.Z2_norm

In [ ]:
datlib.np2dat(
    "data/Z2.dat",
    "m eta zeta h l upsilon eta2 mhj psi0 Z2_norm".split(),
    np.asarray([
        m, eta, zeta, h, l, upsilon, eta2, mhj, psi0, Z2_norm
    ])
)

In [ ]:
# %%bash
# gplt3 -pm3d -U 'upsilon(m,eta)' data/Z2.dat -to data/picts/pict.pdf 

In [ ]:
variables = 1-m, 1-eta
y_data = upsilon
indexes_nans = np.logical_not(np.isnan(h))

def func(_, *p_args, nans=True):    
    result = p_args[0]
    result += p_args[1]*variables[0] + p_args[2]*variables[1]

    if nans:
        return result[indexes_nans].ravel()
    else:
        return result
    

p0 = [0] + 2*[1]
p = opt.curve_fit(func, None, y_data[indexes_nans].ravel(), p0=p0)[0]
p

In [ ]:
# x = 1 - m
# y = 1 - eta

# upsilon = y/2
# eta2 = 1 - 2*y
# mjh2 = 1 - 2*x
# psi0 = -y/2

# Ферромагнетик в несимметричном случае

## Выражение для $Z^{(2)}$

### Аргументы

In [ ]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
s1, s2 = symbase.SigmaSymbol("\\sigma_1"), symbase.SigmaSymbol("\\sigma_2")
t = sp.Symbol("t")

In [ ]:
arg_exp = t**2/(2*l)*hi/hj + s1*t
bound = hj+s2*l

arg_exp = arg_exp.replace(t, t-s1*l*hj/hi).expand()
bound += s1*l*hj/hi

arg_exp = arg_exp.replace(t, t*sp.sqrt(2*l*hj/hi))
bound /= sp.sqrt(2*l*hj/hi)

display(arg_exp)
display(bound)

In [ ]:
expr = bound**2 + (arg_exp - t**2)
expr = expr.expand()
expr -= (hj**2+l**2)/(2*l*hj/hi)
expr = expr.expand()

display(expr)

### Производные $\cal{F}^{(2)}$

In [ ]:
display(Z2sym.E_not_symmetrical(hi, hj, l, 1, 1).diff(l).rewrite(sp.exp))
display(Z2sym.E_not_symmetrical(hi, hj, l, 1, 1).rewrite(sp.exp).diff(l))

In [ ]:
# display(Z2sym.cF2_not_symmetrical(hi, hj, l).diff(hi).rewrite(sp.exp).expand())
# display(Z2sym.cF2_not_symmetrical(hi, hj, l).rewrite(sp.exp).diff(hi).expand())

### Численная проверка

In [ ]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
cF2_diff_hi = sp.lambdify((hi, hj, l), Z2sym.cF2_not_symmetrical(hi, hj, l).diff(hi), Z2sym.num_module)
cF2_diff_hj = sp.lambdify((hi, hj, l), Z2sym.cF2_not_symmetrical(hi, hj, l).diff(hj), Z2sym.num_module)
cF2_diff_l = sp.lambdify((hi, hj, l), Z2sym.cF2_not_symmetrical(hi, hj, l).diff(l), Z2sym.num_module)

args = 1, -2, -1.4
gradient = nd.Gradient(lambda args: Z2num.cF2_not_symmetrical(*args))(args)

print(gradient[0])
print(cF2_diff_hi(*args))
print()
print(gradient[1])
print(cF2_diff_hj(*args))
print()
print(gradient[2])
print(cF2_diff_l(*args))

In [ ]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
sigmas_args = 1, 1
E_diff_hi = sp.lambdify((hi, hj, l), Z2sym.E_not_symmetrical(hi, hj, l, *sigmas_args).diff(hi), Z2sym.num_module)
E_diff_hj = sp.lambdify((hi, hj, l), Z2sym.E_not_symmetrical(hi, hj, l, *sigmas_args).diff(hj), Z2sym.num_module)
E_diff_l = sp.lambdify((hi, hj, l), Z2sym.E_not_symmetrical(hi, hj, l, *sigmas_args).diff(l), Z2sym.num_module)

args = 1, 1.8, 1.4
sigmas_args = list(map(bool, sigmas_args))

gradient = nd.Gradient(lambda args: Z2num.E_not_symmetrical(*args, *sigmas_args))(args)

print(gradient[0])
print(E_diff_hi(*args))
print()
print(gradient[1])
print(E_diff_hj(*args))
print()
print(gradient[2])
print(E_diff_l(*args))

## Моменты

In [ ]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
Z2 = sp.sqrt(2)*(2*sp.pi)**2/sp.sqrt(hi*hj*l)*Z2sym.cF2_not_symmetrical(hi, hj, l)
Z2

### Симметричный случай

In [ ]:
h, l = Z2sym.h, Z2sym.l

In [ ]:
expr_hessian_symmetrical = sp.zeros(3, 3)
for i in range(3):
    for j in range(3):
        arg1, arg2 = [hi, hj, l][i], [hi, hj, l][j]
        expr_hessian_symmetrical[i, j] = Z2.diff(arg1, arg2)/Z2
        expr_hessian_symmetrical[i, j] = expr_hessian_symmetrical[i, j].expand().subs({hi: h, hj: h})

expr = expr_hessian_symmetrical
expr = expr.subs(subs_to_m_and_eta_symmetrical).expand()
expr = expr.subs(subs_to_m_eta_symmetrical)
expr = sp.Matrix(3, 3, lambda i, j: expr[i, j].factor()).expand()

expr[2, 2] = expr[2, 2].subs(subs_to_eta2).factor()

for i in range(3):
    for j in range(3):
        if expr[i, j] != Z2sym.m_eta:
            expr[i, j] = expr[i, j].subs(subs_to_upsilon).expand()

expr

#### Момент $\Psi$

In [ ]:
from sympy.vector import CoordSys3D, Del

N = CoordSys3D("N")
delop = Del()

vhi, vhj, vh3 = sp.symbols("h_1 h_2 h_3")
vh = vhi*N.i + vhj*N.j + vh3*N.k

nk1, nk2, nk3 = sp.symbols("n_{1K} n_{2K} n_{3K}")
nk = nk1*N.i + nk2*N.j + nk3*N.k

In [ ]:
gamma1, gamma2 = sp.symbols("\\gamma_1 \\gamma_2", positive=True)

In [ ]:
v1 = vh + gamma1*nk
v2 = vh + gamma2*nk

v1_par_2 = v1.dot(v2)/v2.magnitude()
v1_per_2 = sp.sqrt(v1.dot(v1) - v1_par_2**2)
v2_magnitude = v2.magnitude()

In [ ]:
def _diff_func(func, args): 
    if len(args) == 0:
        return func

    arg = args[0]
    if isinstance(func, sp.Function):
        func = func.fdiff(arg)
        return _diff_func(func, args[1:])
    if isinstance(func, sp.Subs):
        return sp.Subs(_diff_func(func.args[0], args), *func.args[1:])
    if isinstance(func, sp.Derivative):
        return sp.Derivative(_diff_func(func.args[0], args), *func.args[1:])

In [ ]:
Z = sp.Function("Z")(v1_per_2, v1_par_2, v2_magnitude, l)

def two_diff_func(func, arg1, arg2):
    args = func.args
    result = 0
    for i, arg in enumerate(args):
        result += arg.diff(arg1, arg2).subs({gamma2: 0}).factor()*_diff_func(func, [i+1])

        factor = arg.diff(arg1).subs({gamma2: 0})
        for j, j_arg in enumerate(args):
            term = (factor*j_arg.diff(arg2)).subs({gamma2: 0}).factor()
            result += term*_diff_func(func, [i+1, j+1])
    return result

expr_psi = (two_diff_func(Z, gamma1, gamma1).diff(l) - two_diff_func(Z, gamma1, gamma2)).subs({gamma1: 0, gamma2: 0})
expr_psi = expr_psi.subs(vh.dot(vh), h**2).expand()

def v_per_diff_eval(*args):
    func = args[0]
    diff_args = args[1:]
    per_arg = func.args[0]
    non_per_diff_args = tuple(filter(lambda item: item[0] != per_arg, diff_args))
    per_diff_args = tuple(filter(lambda item: item[0] == per_arg, diff_args))

    if len(per_diff_args) == 0:
        return sp.Derivative(*args)
    
    _, n = per_diff_args[0]

    if n == 2:
        func = func - _diff_func(func, (2, 2))
        func /= 2
    elif n == 1:
        return 0
    else:
        return sp.Derivative(*args)

    if len(non_per_diff_args) > 0:
        func = sp.Derivative(func, *non_per_diff_args)

    return func

expr_psi = expr_psi.replace(sp.Derivative, v_per_diff_eval).doit()

expr_psi = expr_psi.replace(sp.Function("Z"), lambda *args: Z2.subs(dict(zip([hi, hj, l], args[1:])))).doit()
expr_psi = (expr_psi/Z2.subs({hi: h, hj: h})).expand()
expr_psi = expr_psi.factor()
expr_psi = expr_psi.replace(sp.sqrt(h**2), h)
expr_psi

In [ ]:
expr_psi_nk = expr_psi.args[-2]
display(expr_psi_nk)

expr = expr_psi_nk-(3*(vh.dot(nk)**2)-vh.dot(vh))
expr = expr.expand()
expr = expr.subs({nk1**2: 1-nk2**2-nk3**2})
expr.expand()

In [ ]:
expr_psi0 = expr_psi/expr_psi_nk
expr_psi0 = expr_psi0.expand()
expr_psi0 = expr_psi0.replace(h, sp.Abs(h)).replace(sp.Abs(h), h)
expr_psi0 = expr_psi0.subs(subs_to_m_and_eta_symmetrical).expand()
expr_psi0 = expr_psi0.subs(subs_to_m_eta_symmetrical).factor().expand()
expr_psi0 = expr_psi0.subs(subs_to_eta2).factor().expand()
expr_psi0 = expr_psi0.subs(subs_to_upsilon).expand()
expr_psi0

### Численная проверка

#### Определения

In [ ]:
def calc_Z2_integrate(hi, hj, l):
    hjz = np.linalg.norm(hj)
    hiz = hi.dot(hj)/hjz
    hiy = np.sqrt(np.abs(hi.dot(hi) - hiz**2))
    return Z2num.calc_Z2_integrate(hiy, hiz, hjz, l)

In [ ]:
hi = np.asarray([0, 3, 2])
hj = np.asarray([0, 0, 1])
l = 2

print(calc_Z2_integrate(hi, hj, l))
print(Z2num.calc_Z2_integrate(hi[1], hi[2], hj[2], l))
print(Z2num.calc_Z2_integrate(0, np.linalg.norm(hi), np.linalg.norm(hj), l))

In [ ]:
def Z2_not_symmetrical_integrate(args):
    hi, hj, l = args
    return Z2num.calc_Z2_integrate(0, hi, hj, l)

In [ ]:
def calc_Z2_symmetrical_with_nk_integrate(h, l, gamma1, gamma2, nk):
    h = np.asarray([0, 0, h])
    hi = h + gamma1*nk
    hj = h + gamma2*nk
    return calc_Z2_integrate(hi, hj, l)

#### Симметричный случай

In [ ]:
h, l = Z2sym.h, Z2sym.l
f_expr_mh2 = sp.lambdify((h, l), expr_hessian_symmetrical[0, 0], Z2sym.num_module_norm)

args = 1, 1.2
args_not_symmetrical = args[0], args[0], args[1]

moments = nd.Hessdiag(Z2_not_symmetrical_integrate)(args_not_symmetrical)/Z2_not_symmetrical_integrate(args_not_symmetrical)

print(moments[0])
print(moments[1])
print(f_expr_mh2(*args))

In [ ]:
expr_psi

In [ ]:
h, l = Z2sym.h, Z2sym.l
f_expr_psi = sp.lambdify((vhi, vhj, vh3, nk1, nk2, nk3, h, l), expr_psi, Z2sym.num_module_norm)

h, l = 1.4, 1.6
vh = np.asarray([0, 0, h])
nk = np.asarray([0, 1, 1.])
nk /= np.linalg.norm(nk)

print(f_expr_psi(*vh, *nk, h, l))

z2 = calc_Z2_symmetrical_integrate((h, l))

hessian = nd.Hessian(lambda args: calc_Z2_symmetrical_with_nk_integrate(h, l, *args, nk))((0, 0))
hessdiag_l = nd.Derivative(lambda l: nd.Hessdiag(lambda args: calc_Z2_symmetrical_with_nk_integrate(h, l, *args, nk))((0, 0)))(l)

psi = hessdiag_l[0]/z2-hessian[0, 1]/z2
print(psi)

z2 = Z2num.calc_from_coeffs_symmetrical(h, l)
print((1-z2.eta)*z2.m**3*(0.46134-1.3836*(vh/np.linalg.norm(vh)).dot(nk)**2))